# Data wrangling with pandas 2
튜토리얼은 아래의 목차로 구성되어있다.
(본 튜토리얼은 "파이썬 라이브러리를 활용한 데이터 분석"을 기초로 만들어졌다.)

---

1. GroupBy 메카닉
 * 1.1 그룹 간 순회하기
 * 1.2 칼럼 또는 칼럼의 일부만 선택하기
 * 1.3 사전과 Series에서 묶기
 * 1.4 함수로 묶기
 * 1.5 색인 단계로 묶기
2. 데이터 수집
 * 2.1 칼럼에 여러가지 함수 적용하기
 * 2.2 색인되지 않는 형태로 집계된 데이터 반환하기
3. 그룹별 연산과 변형
 * 3.1 apply : 분리-적용-병합
 * 3.2 변위치 분석과 버킷 분석
 * 3.3 예제 : 그룹에 국한된 값으로 누락된 값 채우기
 * 3.4 예제 : 랜덤 표본과 순열
 * 3.5 예제 : 그룹 가중평균과 상관관계
 * 3.6 예제 : 그룹 상의 선형회귀
4. 피벗 테이블과 교차일람표
 * 4.1 교차일람표

## 1. GroupBy 메카닉
그룹의 색인은 다양한 형태가 될 수 있으며, 모두 같은 타입일 필요가 없다. 아래의 목록 중 마지막 세 가지 방법은 모두 객체를 나눌 때 사용할 배열을 생성하기 위한 방법이다.
* 그룹으로 묶을 축과 같은 길이의 리스트나 배열
* DataFrame의 칼럼 이름을 지칭하는 값 
* 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
* 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b','a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,0.769400,-0.530952,a,one
1,-0.841837,0.343161,a,two
2,0.224147,0.613497,b,one
3,0.324268,0.696122,b,two
4,0.043980,-0.734512,a,one


In [5]:
# 위의 예제 데이터에 대해서 key1으로 묶고 각 그룹에서 data1의 평균 계산하기
grouped = df['data1'].groupby(df['key1'])
grouped 
# grouped 변수는 GroupBy 객체로 df['key1]로 참조되는 중간 값에 대한 것 외에는 아무것도 계산되지 않은 객체이다.
# 해당 객체는 그룹 연산을 위해 필요한 모든 정보를 가지고 있기 때문에 각 그룹에 어떤 연산을 적용할 수 있게 해준다.

In [6]:
# key1 칼럼에 있는 유일한 값으로 색인되는 새로운 Series 객체가 생성되었다.
grouped.mean()

key1
a   -0.009486
b    0.274208
Name: data1, dtype: float64

In [7]:
# 여러개의 기준으로 그룹을 만드려면 리스트 형태로 넘긴다.
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means # long format (계층적 색인)

key1  key2
a     one     0.406690
      two    -0.841837
b     one     0.224147
      two     0.324268
Name: data1, dtype: float64

In [8]:
# long format -> wide format by unstack method
means.unstack(level = 'key2')

key2,one,two
key1,,
a,0.406690,-0.841837
b,0.224147,0.324268


In [9]:
# 그룹으로 나눌 때 기준이 되는 것이 길이만 같다면 어떤 배열이든 상관이 없다.
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2005])
df['data1'].groupby([states, years]).mean()

California  2005   -0.841837
            2006    0.224147
Ohio        2005    0.379216
Name: data1, dtype: float64

In [10]:
# 한 그룹으로 묶을 정보는 같은 DataFrame 안에서 주로 찾게 되는데. 이 경우 칼럼 이름(문자열, 숫자 혹은 다른 파이썬 객체)을 넘겨서 그룹의
# 색인으로 사용할 수 있다.
print(df)
df.groupby('key1').mean() # 기본적으로 산술통계를 낼 때, 숫자가 아닌 데이터는 제외된다.

      data1     data2 key1 key2
0  0.769400 -0.530952    a  one
1 -0.841837  0.343161    a  two
2  0.224147  0.613497    b  one
3  0.324268  0.696122    b  two
4  0.043980 -0.734512    a  one


,data1,data2
key1,,
a,-0.009486,-0.307434
b,0.274208,0.654810


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.406690 -0.632732
     two  -0.841837  0.343161
b    one   0.224147  0.613497
     two   0.324268  0.696122

In [12]:
# size 메서드는 그룹의 크기를 담고 있는 Series를 반환한다.
print(df)
df.groupby(['key1', 'key2']).size()

      data1     data2 key1 key2
0  0.769400 -0.530952    a  one
1 -0.841837  0.343161    a  two
2  0.224147  0.613497    b  one
3  0.324268  0.696122    b  two
4  0.043980 -0.734512    a  one


key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 1.1 그룹 간 순회하기
GroupBy 객체는 이터레이션을 지원하는 데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환한다.

In [13]:
df

,data1,data2,key1,key2
0,0.769400,-0.530952,a,one
1,-0.841837,0.343161,a,two
2,0.224147,0.613497,b,one
3,0.324268,0.696122,b,two
4,0.043980,-0.734512,a,one


In [14]:
# 색인이 있으면 튜플의 첫 번째 원소가 색인 값이 된다.
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  0.769400 -0.530952    a  one
1 -0.841837  0.343161    a  two
4  0.043980 -0.734512    a  one
b
      data1     data2 key1 key2
2  0.224147  0.613497    b  one
3  0.324268  0.696122    b  two


In [15]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
     data1     data2 key1 key2
0  0.76940 -0.530952    a  one
4  0.04398 -0.734512    a  one
('a', 'two')
      data1     data2 key1 key2
1 -0.841837  0.343161    a  two
('b', 'one')
      data1     data2 key1 key2
2  0.224147  0.613497    b  one
('b', 'two')
      data1     data2 key1 key2
3  0.324268  0.696122    b  two


In [16]:
# 그룹별 데이터를 사전형으로 바꾸기
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0  0.769400 -0.530952    a  one
 1 -0.841837  0.343161    a  two
 4  0.043980 -0.734512    a  one, 'b':       data1     data2 key1 key2
 2  0.224147  0.613497    b  one
 3  0.324268  0.696122    b  two}

In [17]:
# groupby 메서드는 기본적으로 axis = 0에 대해 그룹을 만드는데, 다른 축으로 그룹을 만드는 것도 가능하다.
# df DataFrame의 dtype에 따라 그룹 묶끼
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [18]:
grouped = df.groupby(df.dtypes, axis = 1) # 변수 칼렁의 dtype에 따라서 그룹 만들기
dict(list(grouped))

{dtype('float64'):       data1     data2
 0  0.769400 -0.530952
 1 -0.841837  0.343161
 2  0.224147  0.613497
 3  0.324268  0.696122
 4  0.043980 -0.734512, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### 1.2 칼럼 또는 칼럼의 일부만 선택하기
DataFrame에서 만든 GroupBy 객체를 칼럼 이름이나 칼럼 이름이 담긴 배열로 색인하면 수집을 위해 해당 칼럼을 선택하게 된다. 색인으로 얻는 객체는 groupby 메서드에 리스트나 배열을 넘기면 DataFrameGroupBy 객체가 생성되고, 리스트나 배열이 아닌 단일 값으로 칼럼 이름 하나만 넘기면 SeriesGroupBy 객체가 된다.

In [19]:
df

,data1,data2,key1,key2
0,0.769400,-0.530952,a,one
1,-0.841837,0.343161,a,two
2,0.224147,0.613497,b,one
3,0.324268,0.696122,b,two
4,0.043980,-0.734512,a,one


In [20]:
print(df[['data2']], type(df[['data2']])) # [[]]는 DataFrame의 일부 칼럼에 접근 할때도 DataFrame 형태를 유지시켜준다.

      data2
0 -0.530952
1  0.343161
2  0.613497
3  0.696122
4 -0.734512 <class 'pandas.core.frame.DataFrame'>


In [21]:
print(df[['data1', 'data2']]) # 중복 칼럼 선택시 ix 메서드를 활용하는 것과 같은 결과이다.
print(df.ix[:,'data1':'data2'])

      data1     data2
0  0.769400 -0.530952
1 -0.841837  0.343161
2  0.224147  0.613497
3  0.324268  0.696122
4  0.043980 -0.734512
      data1     data2
0  0.769400 -0.530952
1 -0.841837  0.343161
2  0.224147  0.613497
3  0.324268  0.696122
4  0.043980 -0.734512


In [22]:
# 위의 df DataFrame에 대해서 data2 칼럼에 대해서만 평균을 구하고 결과를 DataFrame으로 반환하는 경우
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.632732
     two   0.343161
b    one   0.613497
     two   0.696122

In [23]:
print(df.groupby(['key1', 'key2']))
print(df.groupby(['key1', 'key2'])['key1'])
print(df.groupby(['key1', 'key2'])[['key1', 'key2']])

### 1.3 사전과 Series에서 묶기

In [24]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns = ['a', 'b', 'c', 'd', 'e'],
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.047215,0.651827,0.530308,-0.348355,-1.087709
Steve,-1.017203,-1.109841,-0.503955,0.645450,0.389345
Wes,-1.049730,-0.312497,0.952784,-1.568582,0.428277
Jim,0.133677,-0.254271,-0.294644,0.751234,0.324701
Travis,0.361334,-0.685837,0.800414,0.560272,0.710882


In [25]:
people.ix[2:3, ['b', 'c']] = np.nan # NA 값을 몇개 추가한다.
people

,a,b,c,d,e
Joe,0.047215,0.651827,0.530308,-0.348355,-1.087709
Steve,-1.017203,-1.109841,-0.503955,0.645450,0.389345
Wes,-1.049730,NaN,NaN,-1.568582,0.428277
Jim,0.133677,-0.254271,-0.294644,0.751234,0.324701
Travis,0.361334,-0.685837,0.800414,0.560272,0.710882


In [26]:
# 각 칼럼을 나타낼 그룹 목록이 있고, 그룹별로 칼럼의 값을 모두 더한다.
# 아래와 같은 dictionary 객체를 groupby 메서드에 넘긴다.
mapping = {'a' : 'red', 'b' : 'red', 'c' : 'blue', 'd': 'blue', 'e' : 'red', 'f' : 'orange'}

In [27]:
by_column = people.groupby(mapping, axis = 1)
by_column.sum()

,blue,red
Joe,0.181953,-0.388667
Steve,0.141495,-1.737698
Wes,-1.568582,-0.621453
Jim,0.456590,0.204106
Travis,1.360686,0.386380


In [28]:
# Series에 대해서도 위와 같은 기능을 수행할 수 있으며, 로우 색인과 data가 매핑된 형태라고 보면된다.
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [29]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 1.4 함수로 묶기
dictionary나 Series를 사용하는 것에 비해 파이썬 함수를 사용해서 그룹을 매핑하는 것은 좀 더 독창적이며 추상화된 방법이다. 그룹 색인으로 넘긴 함수는 색인 값 하나마다 한 번씩 호출되며, 반환값은 그 그룹의 이름으로 사용된다.

In [30]:
people

,a,b,c,d,e
Joe,0.047215,0.651827,0.530308,-0.348355,-1.087709
Steve,-1.017203,-1.109841,-0.503955,0.645450,0.389345
Wes,-1.049730,NaN,NaN,-1.568582,0.428277
Jim,0.133677,-0.254271,-0.294644,0.751234,0.324701
Travis,0.361334,-0.685837,0.800414,0.560272,0.710882


In [31]:
# 이름의 길이별로 group을 만들어보자
people.groupby(len, axis = 0).sum()

,a,b,c,d,e
3,-0.868839,0.397555,0.235664,-1.165704,-0.334731
5,-1.017203,-1.109841,-0.503955,0.645450,0.389345
6,0.361334,-0.685837,0.800414,0.560272,0.710882


In [32]:
# 내부적으로는 모두 배열로 반환되므로 함수와 배열, 사전 또는 Series를 함께 섞어 쓰는 것도 전혀 문제가 되지 않는다.
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.049730  0.651827  0.530308 -1.568582 -1.087709
  two  0.133677 -0.254271 -0.294644  0.751234  0.324701
5 one -1.017203 -1.109841 -0.503955  0.645450  0.389345
6 two  0.361334 -0.685837  0.800414  0.560272  0.710882

### 1.5 색인 단계로 묶기
계층적으로 색인된 데이터 묶음은 축 색인의 단계 중 하나를 사용해서 편리하게 모을 수 있는 기능을 제공한다. 이 기능을 사용하려면 level 인자를 통해 레벨 번호나 이름을 넘기면 된다.

In [33]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]], names = ['cty', 'tenor'])
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [34]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.975750  0.079220  0.430856 -1.692539 -0.965548
1      0.275469 -0.277090 -0.277273 -0.944237  2.099986
2     -0.122095 -0.545668  0.075468 -0.496827 -0.953389
3      0.410285 -0.361123 -0.165286 -0.369805 -0.490627

In [35]:
hier_df.groupby(level = 'cty', axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 2. 데이터 수집

**<표> 최적화된 groupby 메서드**

| 함수 이름   | 설명                                             |
|-------------|--------------------------------------------------|
| count       | Nu 그룹 내에 NA 값이 아닌 값의 수를 반환한다.    |
| sum         | NA 값이 아닌 값들의 합을 구한다.                 |
| mean        | NA 값이 아닌 값들의 평균 값을 구한다.            |
| median      | NA 값이 아닌 값들의 산술 중간값을 구한다.        |
| std, var    | 편향되지않은 (n-1을 분모로 하는) 표준편차와 분산 |
| min, max    | NA 값이 아닌 값 중 최소값과 최대값               |
| prod        | NA 값이 아닌 값의 곱                             |
| first, last | NA 값이 아닌 값들 중 첫번째값과 마지막 값        |

In [36]:
df

,data1,data2,key1,key2
0,0.769400,-0.530952,a,one
1,-0.841837,0.343161,a,two
2,0.224147,0.613497,b,one
3,0.324268,0.696122,b,two
4,0.043980,-0.734512,a,one


In [37]:
# 내부적으로 GroupBy는 Series를 효과적으로 잘게잘라서 각 조각에 대해 piece.quantile(0.9)를 호출한다.
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.624316
b    0.314256
Name: data1, dtype: float64

In [38]:
# 자신만의 데이터 집계 함수를 사용하고자할 때, 배열의 agg나 aggregate 메서드에 해당 함수를 넘긴다.
# 사용자 정의함수를 적용할 때
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.611237,1.077673
b,0.100121,0.082625


In [39]:
# describe 메서드를 사용한 경우
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.009486 -0.307434
     std    0.806948  0.572551
     min   -0.841837 -0.734512
     25%   -0.398929 -0.632732
     50%    0.043980 -0.530952
     75%    0.406690 -0.093895
     max    0.769400  0.343161
b    count  2.000000  2.000000
     mean   0.274208  0.654810
     std    0.070796  0.058425
     min    0.224147  0.613497
     25%    0.249178  0.634153
     50%    0.274208  0.654810
     75%    0.299238  0.675466
     max    0.324268  0.696122

### 2.1 칼럼에 여러가지 함수 적용하기
Series나 DataFrame에 모든 칼럼을 집계하는 것은 mean이나 std 같은 메서드를 호출하거나 원하는 함수를 aggregate를 통해서 사용하느 ㄴ것이다. 하지만 칼럼에 따라 다른 함수를 사용해서 집계를 수행하거나 여러 개의 함수를 한 번에 적용하고 싶다면 이를 쉽고 간단하게 수행할 수 있다.

In [40]:
# 예제 데이터 생성
example = {'total_bill' : [16.99, 10.34, 21.01, 23.68, 24.59, 25.29], 'tip' : [1.01, 1.66, 3.50, 3.31, 3.61, 4.71],
           'sex' : ['Female', 'Male', 'Male', 'Male', 'Female', 'Male'], 'smoker' : ['No', 'Yes', 'Yes', 'No', 'Yes', 'No'],
           'day' : ['Sun', 'Sun', 'Sun', 'Sun', 'Sun', 'Sun'], 'time' : ['Dinner', 'Dinner', 'Dinner', 'Dinner', 'Dinner', 'Dinner'],
           'size' : [2, 3, 3, 2, 4, 4,]}
example

{'day': ['Sun', 'Sun', 'Sun', 'Sun', 'Sun', 'Sun'],
 'sex': ['Female', 'Male', 'Male', 'Male', 'Female', 'Male'],
 'size': [2, 3, 3, 2, 4, 4],
 'smoker': ['No', 'Yes', 'Yes', 'No', 'Yes', 'No'],
 'time': ['Dinner', 'Dinner', 'Dinner', 'Dinner', 'Dinner', 'Dinner'],
 'tip': [1.01, 1.66, 3.5, 3.31, 3.61, 4.71],
 'total_bill': [16.99, 10.34, 21.01, 23.68, 24.59, 25.29]}

In [41]:
tips = pd.DataFrame(example)
tips

,day,sex,size,smoker,time,tip,total_bill
0,Sun,Female,2,No,Dinner,1.01,16.99
1,Sun,Male,3,Yes,Dinner,1.66,10.34
2,Sun,Male,3,Yes,Dinner,3.50,21.01
3,Sun,Male,2,No,Dinner,3.31,23.68
4,Sun,Female,4,Yes,Dinner,3.61,24.59
5,Sun,Male,4,No,Dinner,4.71,25.29


In [42]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips

,day,sex,size,smoker,time,tip,total_bill,tip_pct
0,Sun,Female,2,No,Dinner,1.01,16.99,0.059447
1,Sun,Male,3,Yes,Dinner,1.66,10.34,0.160542
2,Sun,Male,3,Yes,Dinner,3.50,21.01,0.166587
3,Sun,Male,2,No,Dinner,3.31,23.68,0.139780
4,Sun,Female,4,Yes,Dinner,3.61,24.59,0.146808
5,Sun,Male,4,No,Dinner,4.71,25.29,0.186240


In [43]:
grouped = tips.groupby(['sex', 'smoker'])

In [44]:
# 그룹을 미리 지어놓은 뒤, 원하는 칼럼을 선택하고 함수이름을 agg, aggregate 메서드에 넘긴다.
grouped['tip_pct'].agg('mean')

sex     smoker
Female  No        0.059447
        Yes       0.146808
Male    No        0.163010
        Yes       0.163564
Name: tip_pct, dtype: float64

In [45]:
grouped['tip_pct'].aggregate('mean')

sex     smoker
Female  No        0.059447
        Yes       0.146808
Male    No        0.163010
        Yes       0.163564
Name: tip_pct, dtype: float64

In [46]:
# 함수의 목록이나 함수 이름들을 넘기면 함수 이름을 칼럼 이름으로 하는 DataFrame을 얻는다.
grouped['tip_pct'].agg(['mean', 'std'])

mean       std
sex    smoker                    
Female No      0.059447       NaN
       Yes     0.146808       NaN
Male   No      0.163010  0.032852
       Yes     0.163564  0.004275

In [47]:
# 이름과 함수가 담긴 튜플의 리스트를 넘기면 각 튜플에서 첫 번째 원소는 DataFrame에서 칼럼의 이름으로 사용된다.
# (2개의 튜플을 가지는 리스트가 순서대로 연결된다.)
grouped['tip_pct'].agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.059447       NaN
       Yes     0.146808       NaN
Male   No      0.163010  0.032852
       Yes     0.163564  0.004275

In [48]:
# DataFrame의 칼럼에 여러 개의 함수를 모든 칼럼에 적용할 수 있다.
functions = ['count', 'mean', 'max']
grouped['tip_pct', 'total_bill'].agg(functions)

tip_pct                     total_bill               
                count      mean       max      count    mean    max
sex    smoker                                                      
Female No           1  0.059447  0.059447          1  16.990  16.99
       Yes          1  0.146808  0.146808          1  24.590  24.59
Male   No           2  0.163010  0.186240          2  24.485  25.29
       Yes          2  0.163564  0.166587          2  15.675  21.01

In [49]:
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.059447        NaN       16.990        NaN
       Yes        0.146808        NaN       24.590        NaN
Male   No         0.163010   0.001079       24.485    1.29605
       Yes        0.163564   0.000018       15.675   56.92445

In [50]:
# 칼럼마다 다른 함수를 적용하고 싶다면 agg 메서드에 칼럼 이름에 대응하는 함수가 들어있는 dictionary 객체를 넘긴다.
grouped['tip', 'size'].agg({'tip' : np.max, 'size' : 'sum'})

size   tip
sex    smoker            
Female No         2  1.01
       Yes        4  3.61
Male   No         6  4.71
       Yes        6  3.50

In [51]:
# 단 하나의 칼럼이라도 여러 개의 함수가 적용된다면 DataFrame은 계층적인 칼럼을 가진다.
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

size   tip_pct                              
               sum       min       max      mean       std
sex    smoker                                             
Female No        2  0.059447  0.059447  0.059447       NaN
       Yes       4  0.146808  0.146808  0.146808       NaN
Male   No        6  0.139780  0.186240  0.163010  0.032852
       Yes       6  0.160542  0.166587  0.163564  0.004275

### 2.2 색인되지 않은 형태로 집계된 데이터 반환하기
groupby 메서드에 as_index = False를 넘긴다.

In [52]:
tips.groupby(['sex', 'smoker']).mean()

size   tip  total_bill   tip_pct
sex    smoker                                  
Female No         2  1.01      16.990  0.059447
       Yes        4  3.61      24.590  0.146808
Male   No         3  4.01      24.485  0.163010
       Yes        3  2.58      15.675  0.163564

In [53]:
tips.groupby(['sex', 'smoker'], as_index = False).mean()

,sex,smoker,size,tip,total_bill,tip_pct
0,Female,No,2,1.01,16.990,0.059447
1,Female,Yes,4,3.61,24.590,0.146808
2,Male,No,3,4.01,24.485,0.163010
3,Male,Yes,3,2.58,15.675,0.163564


## 3. 그룹별 연산과 변형
집계는 그룹 연산의 한 종류일 뿐이다. 1차원 배열을 스칼라 값으로 줄여주는 함수를 적용하는, 일반적인 데이터 변형의 한 가지 특수한 경우다. 다양한 그룹 연산을 수행해야할 경우 transform과 apply 메서드를 활용한다.<br>
* transform 메서드의 경우 : 함수가 배열을 스칼라로 바꿔주는 함수일 경우에만 제대로 작동

In [54]:
# DataFrame에서 각 색인별로 그룹의 평균 값을 담기 위한 칼럼을 추가하는 예제 : 방법 1
df

,data1,data2,key1,key2
0,0.769400,-0.530952,a,one
1,-0.841837,0.343161,a,two
2,0.224147,0.613497,b,one
3,0.324268,0.696122,b,two
4,0.043980,-0.734512,a,one


In [55]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,-0.009486,-0.307434
b,0.274208,0.654810


In [56]:
pd.merge(df, k1_means, left_on = 'key1', right_index = True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,0.769400,-0.530952,a,one,-0.009486,-0.307434
1,-0.841837,0.343161,a,two,-0.009486,-0.307434
4,0.043980,-0.734512,a,one,-0.009486,-0.307434
2,0.224147,0.613497,b,one,0.274208,0.654810
3,0.324268,0.696122,b,two,0.274208,0.654810


In [57]:
# DataFrame에서 각 색인별로 그룹의 평균 값을 담기 위한 칼럼을 추가하는 예제 : 방법 2
people

,a,b,c,d,e
Joe,0.047215,0.651827,0.530308,-0.348355,-1.087709
Steve,-1.017203,-1.109841,-0.503955,0.645450,0.389345
Wes,-1.049730,NaN,NaN,-1.568582,0.428277
Jim,0.133677,-0.254271,-0.294644,0.751234,0.324701
Travis,0.361334,-0.685837,0.800414,0.560272,0.710882


In [58]:
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

,a,b,c,d,e
one,-0.213727,-0.017005,0.665361,-0.452222,0.017150
two,-0.441763,-0.682056,-0.399300,0.698342,0.357023


In [59]:
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,-0.213727,-0.017005,0.665361,-0.452222,0.017150
Steve,-0.441763,-0.682056,-0.399300,0.698342,0.357023
Wes,-0.213727,-0.017005,0.665361,-0.452222,0.017150
Jim,-0.441763,-0.682056,-0.399300,0.698342,0.357023
Travis,-0.213727,-0.017005,0.665361,-0.452222,0.017150


In [60]:
# 각 그룹 별 데이터에서 그룹 별 평균을 빼는 예제
def demean(arr):
    return arr - arr.mean()
demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,0.260942,0.668832,-0.135053,0.103867,-1.104859
Steve,-0.575440,-0.427785,-0.104656,-0.052892,0.032322
Wes,-0.836003,NaN,NaN,-1.116361,0.411127
Jim,0.575440,0.427785,0.104656,0.052892,-0.032322
Travis,0.575061,-0.668832,0.135053,1.012494,0.693732


### 3.1 apply : 분리-적용-병합
aggregate와 마찬가지로 transform은 엄격한 요구사항을 갖는 특수한 목적의 함수다. 인자로 넘긴 함수는 반드시 스칼라 값을 생성해야하며 혹은 같은 크기를 가지는 변형된 배열을 생성해야한다. 가장 일반적인 GroupBy 메서드의 목적은 apply인데, apply 메서드는 객체를 여러 조각으로 나누어 전달된 함수를 각 조각에 일괄적으로 적용한 후 이를 다시 합치게 된다.

In [62]:
tips

,day,sex,size,smoker,time,tip,total_bill,tip_pct
0,Sun,Female,2,No,Dinner,1.01,16.99,0.059447
1,Sun,Male,3,Yes,Dinner,1.66,10.34,0.160542
2,Sun,Male,3,Yes,Dinner,3.50,21.01,0.166587
3,Sun,Male,2,No,Dinner,3.31,23.68,0.139780
4,Sun,Female,4,Yes,Dinner,3.61,24.59,0.146808
5,Sun,Male,4,No,Dinner,4.71,25.29,0.186240


In [71]:
# tips 데이터에서 그룹별 상위 2개의 tip_pct 값을 고르는 예제
def top(df, n = 2, column = 'tip_pct'):
    return df.sort_values(by = column)[-n:]

In [73]:
top(tips)

,day,sex,size,smoker,time,tip,total_bill,tip_pct
2,Sun,Male,3,Yes,Dinner,3.50,21.01,0.166587
5,Sun,Male,4,No,Dinner,4.71,25.29,0.186240


In [74]:
# top 함수가 나누어진 DataFrame의 각 부분에 모두 적용이 되었고, pd.concat 함수를 이용해서 하나로 합쳐진 다음 그룹의 이름표가 붙었다.
# 그리하여 결과는 계층적 색인을 가지게 되고 내부 색인은 원본 DataFrame의 색인 값을 가지게 된다.
tips.groupby('smoker').apply(top)

day   sex  size smoker    time   tip  total_bill   tip_pct
smoker                                                              
No     3  Sun  Male     2     No  Dinner  3.31       23.68  0.139780
       5  Sun  Male     4     No  Dinner  4.71       25.29  0.186240
Yes    1  Sun  Male     3    Yes  Dinner  1.66       10.34  0.160542
       2  Sun  Male     3    Yes  Dinner  3.50       21.01  0.166587

In [75]:
# apply 메서드에 넘길 함수가 추가적인 인자를 받는다면 함수 이름 뒤에 붙여서 넘겨주면된다.
tips.groupby(['smoker', 'day']).apply(top, n = 1, column = 'total_bill')

,,,day,sex,size,smoker,time,tip,total_bill,tip_pct
smoker,day,,,,,,,,,
No,Sun,5,Sun,Male,4,No,Dinner,4.71,25.29,0.186240
Yes,Sun,4,Sun,Female,4,Yes,Dinner,3.61,24.59,0.146808


In [77]:
# describe 예제
tips.groupby('smoker')['tip_pct'].describe()

smoker       
No      count    3.000000
        mean     0.128489
        std      0.064146
        min      0.059447
        25%      0.099614
        50%      0.139780
        75%      0.163010
        max      0.186240
Yes     count    3.000000
        mean     0.157979
        std      0.010136
        min      0.146808
        25%      0.153675
        50%      0.160542
        75%      0.163564
        max      0.166587
dtype: float64

In [79]:
# 그룹색인 생략하기
# 앞에서 살펴본 예제에서 반환된 객체들은 원본 객체의 각 조각에 대한 색인과 그룹 키가 계층적 색인으로 사용되는 걸 볼 수 있었다.
# 이런 결과는 groupby 메서드에 group_keys = False를 넘겨서 막을 수 있다.
tips.groupby('smoker').apply(top)

day   sex  size smoker    time   tip  total_bill   tip_pct
smoker                                                              
No     3  Sun  Male     2     No  Dinner  3.31       23.68  0.139780
       5  Sun  Male     4     No  Dinner  4.71       25.29  0.186240
Yes    1  Sun  Male     3    Yes  Dinner  1.66       10.34  0.160542
       2  Sun  Male     3    Yes  Dinner  3.50       21.01  0.166587

In [80]:
tips.groupby('smoker', group_keys = False).apply(top)

,day,sex,size,smoker,time,tip,total_bill,tip_pct
3,Sun,Male,2,No,Dinner,3.31,23.68,0.139780
5,Sun,Male,4,No,Dinner,4.71,25.29,0.186240
1,Sun,Male,3,Yes,Dinner,1.66,10.34,0.160542
2,Sun,Male,3,Yes,Dinner,3.50,21.01,0.166587


### 3.2 변위치 분석과 버킷 분석
pandas의 cut, qcut 함수를 사용해서 선택한 크기 만큼 혹은 표본 변위치에 따라 데이터를 나눌 수 있었다. 이 함수를 groupby와 조합하면 데이터 묶음에 대해 변위치 분석이나 버킷 분석을 아주 쉽게 수행할 수 있다.

In [84]:
frame = pd.DataFrame({'data1' : np.random.randn(1000), 'data2' : np.random.randn(1000)})
frame.head()

,data1,data2
0,0.750526,-1.021263
1,0.653925,-0.875143
2,0.080020,0.668547
3,-0.419739,-0.115031
4,0.491914,0.269782


In [88]:
factor = pd.cut(frame.data1, 4)
factor.head()

<class 'pandas.core.series.Series'>


0     (-0.194, 1.498]
1     (-0.194, 1.498]
2     (-0.194, 1.498]
3    (-1.886, -0.194]
4     (-0.194, 1.498]
Name: data1, dtype: category
Categories (4, object): [(-3.584, -1.886] < (-1.886, -0.194] < (-0.194, 1.498] < (1.498, 3.19]]

In [87]:
frame[:10]

,data1,data2
0,0.750526,-1.021263
1,0.653925,-0.875143
2,0.080020,0.668547
3,-0.419739,-0.115031
4,0.491914,0.269782
5,-0.470547,-1.016144
6,1.044871,-1.369711
7,-0.820424,-0.561767
8,1.100935,0.680332
9,2.269755,1.235414


In [97]:
# cut 메서드는 Factor 객체를 반환하는데, 이 Factor 객체는 바로 groupby로 넘길 수 있다.
def get_stats(group):
    return {'min' : group.min(), 'max' : group.max(),
            'count' : group.count(), 'mean' : group.mean()}
grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack(level = 1)

,count,max,mean,min
data1,,,,
"(-3.584, -1.886]",30.0,1.235209,0.195890,-1.815535
"(-1.886, -0.194]",384.0,3.693600,-0.053622,-3.060557
"(-0.194, 1.498]",501.0,3.155669,-0.003504,-2.907269
"(1.498, 3.19]",85.0,2.339915,-0.091071,-2.169514


### 3.3 예제 : 그룹에 국한된 값으로 누락된 값 채우기
누락된 데이터를 정리할 때면 어떤 경우에는 dropna를 사용해서 데이터를 살펴보고 걸러내기도 한다. 하지만 다른 경우에는 누락된 값을 고정된 값이나 혹은 데이터로부터 도출된 어떤 값으로 채우고 싶을 때도 있다. 이런 경우에 fillna 메서드를 사용한다.

In [101]:
# 누락된 값을 평균값으로 대체하는 예제
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.317352
2         NaN
3    0.723590
4         NaN
5   -2.184400
dtype: float64

In [104]:
s.fillna(s.mean())

0   -0.592721
1   -0.317352
2   -0.592721
3    0.723590
4   -0.592721
5   -2.184400
dtype: float64

In [108]:
# 그룹 별로 채워넣고 싶은 값이 다르다면, 데이터를 그룹으로 나누고 apply 함수를 사용해서 각 그룹에 대해 fillna를 적용한다.
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
grouped_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index = states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.767135
New York     -0.652967
Vermont            NaN
Florida      -0.944964
Oregon       -0.891982
Nevada             NaN
California   -0.147507
Idaho              NaN
dtype: float64

In [109]:
data.groupby(grouped_key).mean()

East    0.056401
West   -0.519745
dtype: float64

In [110]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(grouped_key).apply(fill_mean)

Ohio          1.767135
New York     -0.652967
Vermont       0.056401
Florida      -0.944964
Oregon       -0.891982
Nevada       -0.519745
California   -0.147507
Idaho        -0.519745
dtype: float64

In [111]:
# 그룹에 따라 미리 정의된 다른 값을 채워 넣어야할 경우, 각 그룹은 내부적으로 name이라는 속성을 가지고 있으므로 이를 이용한다.
fill_values = {'East' : 0.5, 'West' : -1}
fill_func = lambda g : g.fillna(fill_values[g.name])
data.groupby(grouped_key).apply(fill_func)

Ohio          1.767135
New York     -0.652967
Vermont       0.500000
Florida      -0.944964
Oregon       -0.891982
Nevada       -1.000000
California   -0.147507
Idaho        -1.000000
dtype: float64

### 3.4 예제 : 랜덤표본과 순열
대용량의 데이터를 몬테카를로 시뮬레이션이나 다른 애플리케이션에서 사용하기 위해 랜덤표본을 뽑아낸다고 하자. 뽑아내는 방법은 여러가지가 있는데, 그 중에서 어떤 방법은 다른 방법에 비해 좀 더 효과적이다. 한 가지 방법은 np.random.permutation(N)에서 처음 k 원소를 선택하는 것이다. 여기서 N은 전체 데이터셋의 크기이고 K는 원하는 표본 크기다.

In [115]:
# 예제 트럼프 카드 덱, 단순히 하기위해 에이스를 1로 취급했다.
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index = cards)

In [116]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [117]:
# 5장의 카드를 뽑아내기위한 함수 정의
def draw(deck, n = 5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

AD      1
QS     10
JH     10
10S    10
AH      1
dtype: int64

In [121]:
# 각 무늬별로 2장의 카드를 무작위로 뽑는 경우, 무늬는 각 카드 이름의 마지막 글자로 나타내고 있으므로, 이를 이용해서 그룹을 나누고 apply 적용
get_suit = lambda card : card[-1]
deck.groupby(get_suit).apply(draw, n = 2)

C  3C      3
   6C      6
D  10D    10
   5D      5
H  8H      8
   7H      7
S  5S      5
   AS      1
dtype: int64

In [122]:
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

2C     2
JC    10
8D     8
2D     2
3H     3
KH    10
8S     8
9S     9
dtype: int64

### 3.5 예제 : 그룹 가중평균과 상관관계
groupby의 나누고 적용하고 합치는 패러다임에서 그룹 가중평균같은 DataFrame에서 칼럼 간의 연산이나 두 Series 간의 연산은 일상적인 일이다.

In [144]:
df = pd.DataFrame({'category' : ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data' : np.random.randn(8),
                   'weights' : np.random.randn(8)})
df

,category,data,weights
0,a,-0.628245,0.123727
1,a,1.133734,-0.380326
2,a,-1.496192,0.728645
3,a,-0.489336,-0.165179
4,b,0.293762,0.192945
5,b,-0.036541,-1.325441
6,b,-0.379811,-1.326942
7,b,-0.716382,1.294246


In [145]:
# category 별 그룹 가중 평균
grouped = df.groupby('category')
get_wavg = lambda g : np.average(g['data'], weights = g['weights'])
grouped.apply(get_wavg)

category
a   -4.947696
b    0.272981
dtype: float64

In [152]:
# 상관관계
df.data.corr(df['weights'])

-0.41891269720943575

In [147]:
# 그룹별 상관관계
get_corr = lambda g : g.data.corr(g.weights)
grouped.apply(get_corr)

category
a   -0.887035
b   -0.337147
dtype: float64

In [148]:
# 그룹 상의 선형회귀 
import statsmodels.api as sm
def regress(data, yvar, xvar):
    Y = data[yvar]
    X = data[xvar]
    X['intecept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

In [150]:
grouped.apply(regress, 'data', ['weights'])

,weights,intecept
category,,
a,-2.022308,-0.214865
b,-0.115013,-0.243246


## 4. 피벗 테이블과 교차일람표

**<표> pivot_table 옵션**

| 함수 이름  | 설명                                                                                                 |
|------------|------------------------------------------------------------------------------------------------------|
| values     | 집계하려는 칼럼 이름 혹은 이름의 리스트, 기본 값으로 숫자 칼럼을 집계한다.                           |
| index      | 만들어지는 피벗 테이블의 로우의 그룹으로 묶을 칼럼 이름이나 그룹 키                                  |
| cols       | 만들어지는 피벗 테이블의 칼럼의 그룹으로 묶을 칼럼 이름이나 그룹 키                                  |
| aggfunc    | 집계 함수나 함수 리스트, 기본 값은 'mean'이고 groupby 컨텍스트 안에서 유효한 어떤 함수라도 가능하다. |
| fill_value | 누락된 값을 대체하기 위한 값                                                                         |
| margins    | 부분합이나 총계를 담기 위한 로우/칼럼을 추가할 지의 여부, 기본 값은 False                            |

In [157]:
tips

,day,sex,size,smoker,time,tip,total_bill,tip_pct
0,Sun,Female,2,No,Dinner,1.01,16.99,0.059447
1,Sun,Male,3,Yes,Dinner,1.66,10.34,0.160542
2,Sun,Male,3,Yes,Dinner,3.50,21.01,0.166587
3,Sun,Male,2,No,Dinner,3.31,23.68,0.139780
4,Sun,Female,4,Yes,Dinner,3.61,24.59,0.146808
5,Sun,Male,4,No,Dinner,4.71,25.29,0.186240


In [158]:
# tips 데이터에서 성별(sex)와 흡연자(smoker) 집단의 평균(pivot_table)
tips.pivot_table(index = ['sex', 'smoker'])

size   tip   tip_pct  total_bill
sex    smoker                                  
Female No         2  1.01  0.059447      16.990
       Yes        4  3.61  0.146808      24.590
Male   No         3  4.01  0.163010      24.485
       Yes        3  2.58  0.163564      15.675

In [159]:
tips.pivot_table(['tip_pct', 'size'], index = ['sex', 'day'], columns = 'smoker')

tip_pct           size     
smoker            No       Yes   No  Yes
sex    day                              
Female Sun  0.059447  0.146808  2.0  4.0
Male   Sun  0.163010  0.163564  3.0  3.0

In [160]:
# margins = True를 넘겨서 부분합을 포함하도록 확장할 수 있는데, 그렇게하면 All 칼럼과 All 로우가 추가되어 단일 줄안에서 그룹 통계를 얻을 수
# 있다.
tips.pivot_table(['tip_pct', 'size'], index = ['sex', 'day'], columns = 'smoker', margins = True)

tip_pct                          size               
smoker            No       Yes       All        No       Yes  All
sex    day                                                       
Female Sun  0.059447  0.146808  0.103127  2.000000  4.000000  3.0
Male   Sun  0.163010  0.163564  0.163287  3.000000  3.000000  3.0
All         0.128489  0.157979  0.143234  2.666667  3.333333  3.0

In [162]:
# 다른 집계 함수를 사용하려면 그냥 aggfunc에 넘기면 되는데, 한 예로 count나 len 함수는 그룹 크기의 교차일람표(총 개수나 빈도)를 반환한다.
tips.pivot_table('tip_pct', index = ['sex', 'smoker'], columns = 'day', aggfunc = len, margins = True)

day            Sun  All
sex    smoker          
Female No      1.0  1.0
       Yes     1.0  1.0
Male   No      2.0  2.0
       Yes     2.0  2.0
All            6.0  6.0

In [165]:
# 만약 어떤 조합이 비어있거나 NA 값이라면 fill_value를 넘길수도 있다.
tips.pivot_table('size', index = ['time', 'sex', 'smoker'], columns = 'day', aggfunc = 'sum')

day                   Sun
time   sex    smoker     
Dinner Female No        2
              Yes       4
       Male   No        6
              Yes       6

In [164]:
# 만약 어떤 조합이 비어있거나 NA 값이라면 fill_value를 넘길수도 있다.
tips.pivot_table('size', index = ['time', 'sex', 'smoker'], columns = 'day', aggfunc = 'sum', fill_value = 0)

day                   Sun
time   sex    smoker     
Dinner Female No        2
              Yes       4
       Male   No        6
              Yes       6

### 4.1 교차일람표
pivot_table 메서드를 활용할 수도 있지만 pandas.crosstab 함수가 훨씬 더 편리하다.

In [169]:
tips

,day,sex,size,smoker,time,tip,total_bill,tip_pct
0,Sun,Female,2,No,Dinner,1.01,16.99,0.059447
1,Sun,Male,3,Yes,Dinner,1.66,10.34,0.160542
2,Sun,Male,3,Yes,Dinner,3.50,21.01,0.166587
3,Sun,Male,2,No,Dinner,3.31,23.68,0.139780
4,Sun,Female,4,Yes,Dinner,3.61,24.59,0.146808
5,Sun,Male,4,No,Dinner,4.71,25.29,0.186240


In [173]:
pd.crosstab(tips.sex, tips.smoker, margins = True)

smoker,No,Yes,All
sex,,,
Female,1,1,2
Male,2,2,4
All,3,3,6


In [174]:
# crosstab 함수의 처음 두 인자는 배열이나 Series, 혹은 배열의 리스트가 될 수 있다.
pd.crosstab([tips.time, tips.day], tips.smoker, margins = True)

,smoker,No,Yes,All
time,day,,,
Dinner,Sun,3,3,6
All,,3,3,6
